In [14]:

from sklearn import metrics
import numpy as np

# 1단계 csv ---> 데이터 프레임으로 변환

import pandas as pd
import seaborn as sns

df = pd.read_csv("d:\\data\\csv\\train.csv")

# 컬럼이 모두다 출력될 수 있도록 출력할 열의 개수 한도를 늘리기
pd.set_option('display.max_columns',15)
# 2단계 결측치 확인하고 제거하거나 치환한다.
# 2.1 타이타닉 데이터 프레임의 자료형을 확인한다.



mask4 = (df.Age<10) | (df.Sex=='female') 
df['child_women']=mask4.astype(int)

print ( df.columns)


df['title'] = df.Name.str.split('[.,]').str[1].str.strip()
df2 =df.groupby('title')['Age'].mean().round(1).reset_index()
df = pd.merge(df,df2,on='title')
df['Age_x'].fillna(df.Age_y, inplace=True)

# 2.2 결측치(NaN) 을 확인한다.
# 2.3 deck 컬럼과 embark_town 컬럼을 삭제한다.
# 설명 : deck 결측치가 많아서 컬럼을 삭제해야함.
# embark 와 embark_town 이 같은 데이터여서 embark 컬럼을 삭제해야함

rdf = df.drop(['PassengerId','Cabin','Name','Ticket'], axis =1)
print(rdf)

# 2.4 age(나이) 열에 나이가 없는 모든행을 삭제한다.
# 데이터가 한개라도 없으면 drop 해라 (how = 'any')
# 모든 데이터가 없으면 drop 해라 (how = 'all')
# 나이의 결측치를 최빈값으로 치환한다.

#print(rdf.shape)

#rdf = rdf.dropna( subset=['Age'], how='all', axis=0)
#rdf['Age']=rdf['Age'].fillna(method='ffill', inplace=True)

#most_freq = rdf['Age'].value_counts(dropna=True).idxmax()  
#rdf['Age'].fillna(rdf.Age_y, inplace=True)
print(rdf.shape)

# 2.5 embark 열의 NaN 값을 승선도시중 가장 많이 출현한 값으로 치환하기
most_freq = rdf['Embarked'].value_counts().idxmax()
rdf['Embarked'].fillna(most_freq, inplace = True)
print(rdf)


# 2.6 fare 의 이상치를 제거합니다.

local_std = rdf.Fare.std() * 5
print(local_std)
rdf = rdf[:][ rdf['Fare'] < local_std  ]

# 3단계 범주형 데이터를 숫자형으로 변환하기
# 3.1 feature selection (분석에 필요한 속성을 선택)
#ndf = rdf[['Survived','Pclass','Sex','Age','Sibsp','Parch','Embarked','child_women']]

ndf = rdf

# 선택된 컬럼중 2개(sex, embarked) 가 범주형이다.
#3.2 범주형 데이터를 숫자로 변환하기(원핫 인코딩)

gender = pd.get_dummies(ndf['Sex'])
ndf = pd.concat([ndf,gender], axis= 1)
onehot_embarked = pd.get_dummies(ndf['Embarked'])
ndf = pd.concat([ndf,onehot_embarked],axis=1)
ndf.drop(['Sex','Embarked'], axis=1, inplace = True)

print(ndf.columns)


# 4단계 정규화
# 4.1 독립변수와 종속변수(라벨) 을 지정한다.
# survived  pclass   age  sibsp  parch  female  male  C  Q  S
#   라벨                       데이터
# 종속변수                     독립변수

print(ndf.columns)
x = ndf[ ['Fare', 'Pclass', 'Age_x' ,'SibSp', 'Parch' ,'female' ,'male', 'C' ,'Q' ,'S',
          'child_women'] ]

#x = ndf[ ['Fare', 'Pclass', 'Age' ,'SibSp', 'Parch' ,'female' ,'male', 'child_women', 'Parch'] ]

y = ndf['Survived'] # 종속변수
print(x)




# 4.2 독립변수들을 정규화 한다.
from sklearn import preprocessing
X = preprocessing.StandardScaler().fit(x).transform(x)



# 7단계 테스트 데이터로 예측을 한다.

x_ktest = pd.read_csv("d:\\data\\csv\\test.csv")
# 2단계 결측치 확인하고 제거하거나 치환한다.
# 2.1 타이타닉 데이터 프레임의 자료형을 확인한다.


mask4 = (x_ktest.Age<10) | (x_ktest.Sex=='female') 
x_ktest['child_women']=mask4.astype(int)
print ( x_ktest.columns)

# 2.2 결측치(NaN) 을 확인한다.
x_ktest['title'] = x_ktest.Name.str.split('[.,]').str[1].str.strip()
x_ktest2 =x_ktest.groupby('title')['Age'].mean().round(1).reset_index()
x_ktest = pd.merge(x_ktest,x_ktest2,on='title')
x_ktest['Age_x'].fillna(df.Age_y, inplace=True)
# 2.3 deck 컬럼과 embark_town 컬럼을 삭제한다.
# 설명 : deck 결측치가 많아서 컬럼을 삭제해야함.
#  embark 와 embark_town 이 같은 데이터여서 embark 컬럼을 삭제해야함

rdf_x_ktest = x_ktest.drop(['PassengerId','Cabin','Name','Ticket'], axis =1)
print(rdf_x_ktest)


# 2.4 age(나이) 열에 나이가 없는 모든행을 삭제한다.
# 데이터가 한개라도 없으면 drop 해라 (how = 'any')

# 모든 데이터가 없으면 drop 해라 (how = 'all')
#rdf_x_ktest = rdf_x_ktest.dropna( subset=['Age'], how='any', axis=0)

# most_freq = rdf_x_ktest['Age'].value_counts(dropna=True).idxmax()  
# rdf_x_ktest['Age'].fillna(most_freq, inplace=True)
# print(rdf_x_ktest.shape)



# 2.5 embark 열의 NaN 값을 승선도시중 가장 많이 출현한 값으로 치환하기
most_freq = rdf_x_ktest['Embarked'].value_counts().idxmax()
rdf_x_ktest['Embarked'].fillna(most_freq, inplace = True)
print(rdf_x_ktest)


# 2.6 fare 열의 NaN 값을 요금중 가장 많이 출현한 값으로 치환하기
most_freq = rdf_x_ktest['Fare'].value_counts().idxmax()
rdf_x_ktest['Fare'].fillna(most_freq, inplace = True)
print(rdf_x_ktest)


# 3단계 범주형 데이터를 숫자형으로 변환하기
# 3.1 feature selection (분석에 필요한 속성을 선택)
#ndf = rdf[['Survived','Pclass','Sex','Age','Sibsp','Parch','Embarked','child_women']]

ndf_x_ktest = rdf_x_ktest

# 선택된 컬럼중 2개(sex, embarked) 가 범주형이다.
#3.2 범주형 데이터를 숫자로 변환하기(원핫 인코딩)
gender = pd.get_dummies(ndf_x_ktest['Sex'])
ndf_x_ktest = pd.concat([ndf_x_ktest,gender], axis= 1)
onehot_embarked = pd.get_dummies(ndf_x_ktest['Embarked'])
ndf_x_ktest = pd.concat([ndf_x_ktest,onehot_embarked],axis=1)
ndf_x_ktest.drop(['Sex','Embarked'], axis=1, inplace = True)


# 4단계 정규화
# 4.1 독립변수와 종속변수(라벨) 을 지정한다.
# survived  pclass   age  sibsp  parch  female  male  C  Q  S
#   라벨                       데이터
# 종속변수                     독립변수
print(ndf_x_ktest.columns)
x = ndf_x_ktest[ ['Fare','Pclass', 'Age_x' ,'SibSp', 'Parch' ,'female' ,'male', 'C' ,'Q' ,'S',
                       'child_women'] ]
#x = ndf_x_ktest[ ['Fare','Pclass', 'Age' ,'SibSp', 'Parch' ,'female' ,'male', 'child_women','Parch'] ]


print ( x.isnull().sum( axis=0)  )  


#y = ndf_x_ktest['Survived'] # 종속변수

# 4.2 독립변수들을 정규화 한다.
from sklearn import preprocessing
X_test = preprocessing.StandardScaler().fit(x).transform(x)
print(len(X))
print(X.shape)
print(X_test.shape)


from  sklearn.ensemble   import  RandomForestClassifier 

tree_model = RandomForestClassifier( n_estimators=200, oob_score=False, 
                                     random_state=14)  


tree_model.fit( X, y )

#print ( tree_model.oob_score_)


# 7단계 테스트 데이터로 예측을 한다.

y_hat = tree_model.predict( X_test )
print(y_hat)


for  i,a  in  enumerate(y_hat):
    print (str(i+892) + ',' + str(a))

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked', 'child_women'],
      dtype='object')
     Survived  Pclass     Sex  Age_x  SibSp  Parch     Fare Embarked  \
0           0       3    male   22.0      1      0   7.2500        S   
1           0       3    male   35.0      0      0   8.0500        S   
2           0       3    male   32.4      0      0   8.4583        Q   
3           0       1    male   54.0      0      0  51.8625        S   
4           0       3    male   20.0      0      0   8.0500        S   
..        ...     ...     ...    ...    ...    ...      ...      ...   
886         1       1    male   56.0      0      0  35.5000        C   
887         0       1    male   60.0      0      0  26.5500        S   
888         0       1    male   70.0      1      1  71.0000        S   
889         1       1  female   33.0      0      0  86.5000        S   
890         0       1    male   38.0    

[0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 1 1 0 0 0 0 1 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 1 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0
 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 0 1 1 1 1
 1 1 1 0 1 1 1 0 1 1 1 0 1 1 1 1 0 0 1 1 1 1 0 1 0 1 0 1 1 0 1 1 1 0 1 1 1
 1 1 1 0 0 1 1 0 0 1 1 1 1 1 0 1 1 1 0 1 1 0 1 0 1 1 1 1 1 1 1 1 0 0 1 1 0
 1 0 0 1 1 1 0 1 1 0 1 1 0 1 1 1 0 1 0 1 1 0 0 1 1 0 1 0 1 1 1 0 1 1 0 0 0
 0 0 0 0 0 0 0 0 0 0 1]
892,0
893,0
894,0
895,0
896,0
897,0
898,0
899,0
900,0
901,0
902,1
903,1
904,0
905,0
906,0
907,0
908,0
909,0
910,0
911,0
912,1
913,0
914,1
915,0
916,0
9

In [6]:
df.Name.str.split('[.,]').str[1].str.strip()

0        Mr
1       Mrs
2      Miss
3       Mrs
4        Mr
       ... 
886     Rev
887    Miss
888    Miss
889      Mr
890      Mr
Name: Name, Length: 891, dtype: object